In [3]:
import requests
import json
import pandas as pd
import plotly.express as px
import plotly.utils

url = "http://149.125.110.45:5000/transactions/time/metrics"
response = requests.get(url)

if response.status_code == 200:
    data = json.loads(response.content)
    
    fraudPercent = []
    hour = []
    for result in data['results']:
        if '' in result:
            del result['']
        fraudPercent.append(result["fraud_percentage"])
        hour.append(result["hour"])
        
    df = pd.DataFrame({"fraud_percentage": fraudPercent, "hour": hour})
    fig = px.line(df, x="hour", y="fraud_percentage")
    fig.write_html("plot.html")  # save the plot to an HTML file
else:
    print('Error in getting response from the url')


In [5]:
import requests
import json
import pandas as pd
import plotly.express as px

def get_state():
    url = "http://149.125.110.45:5000/transactions"
    response = requests.get(url)

    if response.status_code == 200:
        data = json.loads(response.content)
        
        transactions = []
        for transaction in data['transactions']:
            if '' in transaction:
                del transaction['']
            transactions.append((transaction["state"], int(transaction["is_fraud"])))
        
        # create a dataframe with state and fraud status columns
        df = pd.DataFrame(transactions, columns=['state', 'is_fraud'])
        
        # group by state and calculate the number of fraudulent transactions
        fraud_df = df.groupby('state')['is_fraud'].sum().reset_index()
        
        # get state codes to use in the choropleth map
        state_codes = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')['code']
        
        # merge state codes with fraud data
        final_df = pd.merge(fraud_df, state_codes, left_on='state', right_on='state')
        
        # create choropleth map
        fig = px.choropleth(final_df, 
                            locations='code', 
                            locationmode="USA-states", 
                            color='is_fraud',
                            color_continuous_scale="Viridis_r", 
                            scope="usa")
        
        fig.show()
